In [4]:
import os.path
import pandas as pd
from googleapiclient.discovery import build
# apschecule
# slack 연동

# SQL CREATE *
# googleapiclient to SQL


DEVELOPER_KEY = ''
YOUTUBE_API_SERVICE_NAME='youtube'
YOUTUBE_API_VERSION='v3'

def get_channel_id(youtube):
    search_response=youtube.search().list(
        q="백종원 PAIK JONG WON", # 채널 검색
        part="snippet",
        maxResults=50,
    ).execute()

    # * 백종원 PAIK JONG WON channel_id 가져오기
    channel_id = search_response['items'][0]['snippet']['channelId']

    return channel_id

# def get_channel_stats(youtube, channel_id):
#     # * 통계 가져오기
#     # * 'snippet': {'title': '백종원 PAIK JONG WON', 'description': '', 'customUrl': '@paik_jongwon', 'publishedAt': '2018-03-05T02:51:11Z', 'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/J3OI66Bc7T3nheyKJKAkXR6tb-_bpCsoRMTFoslOBCXI3DpVY8eFY4LZWzww3BEgkRjMOEoI=s88-c-k-c0x00ffffff-no-rj', 'width': 88, 'height': 88}, 'medium': {'url': 'https://yt3.ggpht.com/J3OI66Bc7T3nheyKJKAkXR6tb-_bpCsoRMTFoslOBCXI3DpVY8eFY4LZWzww3BEgkRjMOEoI=s240-c-k-c0x00ffffff-no-rj', 'width': 240, 'height': 240}, 'high': {'url': 'https://yt3.ggpht.com/J3OI66Bc7T3nheyKJKAkXR6tb-_bpCsoRMTFoslOBCXI3DpVY8eFY4LZWzww3BEgkRjMOEoI=s800-c-k-c0x00ffffff-no-rj', 'width': 800, 'height': 800}}, 'localized': {'title': '백종원 PAIK JONG WON', 'description': ''}, 'country': 'KR'}
#     # * 'contentDetails': {'relatedPlaylists': {'likes': '', 'uploads': 'UUyn-K7rZLXjGl7VXGweIlcA'}}
#     # * 'statistics': {'viewCount': '884849112', 'subscriberCount': '5770000', 'hiddenSubscriberCount': False, 'videoCount': '518'}
#     request = youtube.channels().list(
#         part="snippet,contentDetails,statistics",
#         id=channel_id
#     )

#     response = request.execute()
    
#     return response['items'] # * include 'snippet', 'contentDetails', 'statistics'

def get_video_list(youtube, channel_id):
    # * channel_id 의 playlists 를 maxResults 20개 가져온다.
    video_list = []
    playlists = youtube.playlists().list(
        channelId=channel_id,
        part='snippet',
        maxResults=20).execute()
    
    # * 내가 필요한 쿠킹로그 데이터는 [10] 에 위치해 있다.
    cooking_logs_id = playlists['items'][10]['id']

    # * 쿠킹로그의 리스트 (playlistId=cooking_logs_id,) 를 50개씩 가져온다.
    request = youtube.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=cooking_logs_id,
        maxResults=50
    )
    next_page = True
    while next_page:
        response = request.execute()
        data = response['items']

        for video in data:
            video_id = video['contentDetails']['videoId']
            # * video_id 를 중복없이 video_list 에 삽입한다.
            if video_id not in video_list:
                video_list.append(video_id)

        # Do we have more pages?
        if 'nextPageToken' in response.keys():
            next_page = True
            request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                playlistId=cooking_logs_id,
                pageToken=response['nextPageToken'], # * google-api-python-client 에서 지원하는 nextPageToken
                maxResults=50
            )
        else:
            next_page = False # * 'nextPageToken' in response.keys() 가 없다면 End

    return video_list

def get_video_details(youtube, video_list):
    # * details 가 필요한 이유는 view_count, like_count, dislike_count, comment_count 를 가져와야 하기 때문
    stats_list=[]

    # Can only get 50 videos at a time.
    # * 0~50 까지의 데이터를 가져온다. 323개의 데이터 까지 6번 반복
    for i in range(0, len(video_list), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=video_list[i:i+50]
        )

        data = request.execute()
        for video in data['items']:
            url_pk=video['id']
            title=video['snippet']['title']
            description=video['snippet']['description']
            view_count=video['statistics'].get('viewCount',0)
            like_count=video['statistics'].get('likeCount',0)
            published=video['snippet']['publishedAt']
            stats_dict=dict(url_pk=url_pk, title=title, description=description, published=published, view_count=view_count, like_count=like_count)
            stats_list.append(stats_dict)

    return stats_list

def create_csv_and_model(video_data):
    df=pd.DataFrame(video_data)
    df.to_csv("/Users/cslee/vscode/self-dining-backend/csv/백종원_쿠킹로그.csv")


# if os.path.exists('csv/백종원_쿠킹로그.csv'):
    # [{'message': 'The request cannot be completed because you have exceeded your quota.', 'domain': 'youtube.quota', 'reason': 'quotaExceeded'}]
    # https://stackoverflow.com/questions/47408723/youtube-quotas-exceeded

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey=DEVELOPER_KEY)
channel_id = get_channel_id(youtube)
video_list = get_video_list(youtube, channel_id=channel_id)
video_data = get_video_details(youtube, video_list=video_list)
create_csv_and_model(video_data=video_data)
# else:
    # print('pass')